In [1]:
# Import
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision

In [2]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyperparameters
in_channel = 3
num_classes = 10
learning_rate = 1e-3
batch_size = 1024
num_epochs = 5

In [18]:
# Load pretrain model & modify it
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    
    def forward(self, x):
        return x

# model = torchvision.models.vgg16(pretrained=False)
# model.avgpool = Identity()
# model.classifier = nn.Linear(512, 10)

model = torchvision.models.vgg16(pretrained=True)
for param in model.parameters():
    param.requires_grad = False 

model.avgpool = Identity()
model.classifier = nn.Sequential(nn.Linear(512, 100),
                                 nn.ReLU(),
                                 nn.Linear(100, 10))
model.to(device=device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
# Load Data
train_dataset = datasets.CIFAR10(root='dataset/', train=True, \
                                 transform=transforms.ToTensor(),
                                 download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, \
                          shuffle=True)

170499072it [00:25, 6813762.38it/s]                               


Extracting dataset/cifar-10-python.tar.gz to dataset/


In [7]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [19]:
# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())
        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f'Cost as epoch {epoch} is {sum(losses)/len(losses)}')

Cost as epoch 0 is 2.3149933377090766
Cost as epoch 1 is 2.3150027917355906
Cost as epoch 2 is 2.3149981547375114
Cost as epoch 3 is 2.3150264438317745
Cost as epoch 4 is 2.3150054143399608


In [15]:
from tqdm import tqdm
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training dataset")
    else:
        print("Checking accuracy on test dataset")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        acc = round(float(num_correct)/float(num_samples)*100, 2)
        print(f'Got {num_correct} / {num_samples} with accuracy \
                {acc}')

In [20]:
check_accuracy(train_loader, model)
# check_accuracy(test_loader, model)

Checking accuracy on training dataset


100%|██████████| 49/49 [00:04<00:00, 10.10it/s]

Got 3627 / 50000 with accuracy                 7.25
